In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 
import warnings
warnings.filterwarnings("ignore")
pd.set_option("display.max_columns" , 50)
from collections import Counter
from sklearn.metrics import accuracy_score, f1_score,confusion_matrix,classification_report,roc_auc_score,roc_curve,auc
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import recall_score, precision_score
from sklearn.ensemble import RandomForestClassifier

In [2]:
df_train = pd.read_csv(r"C:\Users\admin\Desktop\kaggle\train.csv")
df_train.head()

,sku,national_inv,lead_time,in_transit_qty,forecast_3_month,forecast_6_month,forecast_9_month,sales_1_month,sales_3_month,sales_6_month,sales_9_month,min_bank,potential_issue,pieces_past_due,perf_6_month_avg,perf_12_month_avg,local_bo_qty,deck_risk,oe_constraint,ppap_risk,stop_auto_buy,rev_stop,went_on_backorder
0,2,4,2,0,0,0,0,0,0,0,0,0,0,0,0.18,0.31,0,1,0,0,1,0,0
1,5,2,16,0,0,0,0,0,0,0,0,1,0,0,0.31,0.40,0,0,0,0,1,0,0
2,7,4,4,0,0,0,0,0,0,0,0,0,0,0,0.73,0.78,0,0,0,0,1,0,0
3,9,0,9,0,1,1,1,0,0,0,0,0,0,0,0.86,0.69,0,0,0,0,1,0,0
4,10,300,8,0,0,0,0,0,40,100,122,7,0,0,0.00,0.00,0,0,0,0,1,0,0


In [3]:
df_train.shape

(58919, 23)

In [4]:
X = df_train.drop(['went_on_backorder','sku'],axis=1)
y = df_train["went_on_backorder"]

In [6]:
### VIF 

def vif_cal(input_data, dependent_col):
    vif_df = pd.DataFrame( columns = ['Var', 'Vif'])
    x_vars=input_data.drop([dependent_col], axis=1)
    xvar_names=x_vars.columns
    for i in range(0,xvar_names.shape[0]):
        y=x_vars[xvar_names[i]] 
        x=x_vars[xvar_names.drop(xvar_names[i])]
        rsq=sm.OLS(y,x).fit().rsquared  
        vif=round(1/(1-rsq),2)
        vif_df.loc[i] = [xvar_names[i], vif]
    return vif_df.sort_values(by = 'Vif', axis=0, ascending=False, inplace=False)

In [7]:
df_train.columns

Index(['sku', 'national_inv', 'lead_time', 'in_transit_qty',
       'forecast_3_month', 'forecast_6_month', 'forecast_9_month',
       'sales_1_month', 'sales_3_month', 'sales_6_month', 'sales_9_month',
       'min_bank', 'potential_issue', 'pieces_past_due', 'perf_6_month_avg',
       'perf_12_month_avg', 'local_bo_qty', 'deck_risk', 'oe_constraint',
       'ppap_risk', 'stop_auto_buy', 'rev_stop', 'went_on_backorder'],
      dtype='object')

In [9]:
# Vif
vif_cal(input_data=df_train.drop(['sku'], axis=1), dependent_col='went_on_backorder')

,Var,Vif
14,perf_12_month_avg,112.07
13,perf_6_month_avg,104.76
8,sales_6_month,91.08
4,forecast_6_month,85.70
5,forecast_9_month,58.59
7,sales_3_month,48.71
9,sales_9_month,40.52
19,stop_auto_buy,15.54
3,forecast_3_month,14.53
6,sales_1_month,11.12


In [11]:
X1 = df_train.drop(['sku',"went_on_backorder"], axis=1)

y1=df_train["went_on_backorder"]
X1.drop(['deck_risk','oe_constraint','ppap_risk','rev_stop','stop_auto_buy'],axis = 1,inplace = True)

X1.drop(['min_bank'],axis = 1,inplace = True)
X1.drop(['local_bo_qty'],axis = 1,inplace = True)
X1.drop(['potential_issue'],axis = 1,inplace = True)





In [12]:
X1.head()

,national_inv,lead_time,in_transit_qty,forecast_3_month,forecast_6_month,forecast_9_month,sales_1_month,sales_3_month,sales_6_month,sales_9_month,pieces_past_due,perf_6_month_avg,perf_12_month_avg
0,4,2,0,0,0,0,0,0,0,0,0,0.18,0.31
1,2,16,0,0,0,0,0,0,0,0,0,0.31,0.40
2,4,4,0,0,0,0,0,0,0,0,0,0.73,0.78
3,0,9,0,1,1,1,0,0,0,0,0,0.86,0.69
4,300,8,0,0,0,0,0,40,100,122,0,0.00,0.00


In [13]:
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier,AdaBoostClassifier,ExtraTreesClassifier,VotingClassifier,BaggingClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn import model_selection
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import MultinomialNB
from xgboost import XGBClassifier
from sklearn.svm import SVC
from mlxtend.classifier import StackingCVClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from mlxtend.classifier import StackingClassifier
NBmodle = MultinomialNB()

In [16]:
RFm=RandomForestClassifier(criterion='gini',min_samples_leaf=1, max_depth=13,n_estimators=200)
RFm_1=RandomForestClassifier(criterion='gini',min_samples_leaf=1, max_depth=15,n_estimators=400)
# models.append(('GradientBoostingClassifier', GradientBoostingClassifier()))
# models.append(('AdaBoostClassifier', AdaBoostClassifier()))
# models.append(('MNBmodle', MultinomialNB()))
# models.append(('ExtraTreesClassifier', ExtraTreesClassifier()))
# mixmodels.append(('XGBClassifier', XGBClassifier(learning_rate=0.15, max_depth=14, n_estimators =200)))
# XGBm_1=XGBClassifier(eval_metric='error',max_depth=11,alpha=0.6,subsample=0.65,colsample_bytree=0.7,colsample_bylevel = 0.8675692743597421,objective='binary:logistic',use_label_encoder=False,learning_rate=0.009,n_estimators=200,min_child_weight = 366,tree_method='hist',n_jobs=-1)
XGBm=XGBClassifier(eval_metric='error',max_depth=10,subsample=0.8,colsample_bytree=0.5,learning_rate=0.1,objective='binary:logistic',use_label_encoder=False,n_estimators=400,tree_method='hist')
XGBm_1=XGBClassifier(eval_metric='error',max_depth=12,subsample=1,colsample_bytree=0.8,learning_rate=0.2,objective='binary:logistic',use_label_encoder=False,n_estimators=600,tree_method='hist')
ADAm=AdaBoostClassifier(DecisionTreeClassifier(max_depth=9, min_samples_split=10, min_samples_leaf=5),
                         algorithm="SAMME",
                         n_estimators=600, learning_rate=1)
#ADAm_1=AdaBoostClassifier(DecisionTreeClassifier(max_depth=8, min_samples_split=10, min_samples_leaf=3),
#                          algorithm="SAMME",
#                          n_estimators=600, learning_rate=0.8)
GBm=GradientBoostingClassifier(max_depth=13, min_samples_split=10, min_samples_leaf=2,n_estimators=400,learning_rate=0.1,subsample=0.7)
ETm=ExtraTreesClassifier()
lr = LogisticRegression()
 
stackm = StackingClassifier(classifiers=[RFm, RFm_1,XGBm, XGBm_1,ADAm,GBm,ETm],use_probas=True,average_probas=False, meta_classifier=lr)
for clf, label in zip([RFm,RFm_1 ,XGBm, XGBm_1, ADAm, GBm, ETm, stackm], 
 
                      ['RF', 
                       'RF', 
                       'XGB',
                       
                       'XGB',
                       
                       'ADA',
                       
                       'GB',
                       
                       'ET',
 
                       'Stack']):
 
 
 
    scores = model_selection.cross_val_score(clf, X1, y1,  cv=5, scoring='accuracy')
 
    print("Accuracy: %0.4f (+/- %0.4f) [%s]" % (scores.mean(), scores.std(), label))
    
    scores = model_selection.cross_val_score(clf, X1, y1,  cv=5, scoring='f1_macro')
 
    print("F1: %0.4f (+/- %0.4f) [%s]" % (scores.mean(), scores.std(), label))
    
    

Accuracy: 0.9205 (+/- 0.0024) [RF]
F1: 0.8406 (+/- 0.0037) [RF]
Accuracy: 0.9233 (+/- 0.0021) [RF]
F1: 0.8467 (+/- 0.0042) [RF]
Accuracy: 0.9264 (+/- 0.0025) [XGB]
F1: 0.8569 (+/- 0.0054) [XGB]
Accuracy: 0.9257 (+/- 0.0022) [XGB]
F1: 0.8576 (+/- 0.0048) [XGB]
Accuracy: 0.9258 (+/- 0.0014) [ADA]
F1: 0.8568 (+/- 0.0040) [ADA]
Accuracy: 0.9277 (+/- 0.0019) [GB]
F1: 0.8621 (+/- 0.0049) [GB]
Accuracy: 0.9249 (+/- 0.0022) [ET]
F1: 0.8483 (+/- 0.0055) [ET]
Accuracy: 0.9299 (+/- 0.0023) [Stack]
F1: 0.8645 (+/- 0.0062) [Stack]


In [27]:
df_train = pd.read_csv(r"C:\Users\admin\Desktop\kaggle\train.csv")

# y1 = df['went_on_backorder']

In [28]:
X1 = df_train.drop(['went_on_backorder','stop_auto_buy',"sku",'rev_stop','oe_constraint','potential_issue'],axis=1)

In [ ]:

RFm=RandomForestClassifier(criterion='gini',min_samples_leaf=1, max_depth=15,n_estimators=400)
# models.append(('GradientBoostingClassifier', GradientBoostingClassifier()))
# models.append(('AdaBoostClassifier', AdaBoostClassifier()))
# models.append(('MNBmodle', MultinomialNB()))
# models.append(('ExtraTreesClassifier', ExtraTreesClassifier()))
# mixmodels.append(('XGBClassifier', XGBClassifier(learning_rate=0.15, max_depth=14, n_estimators =200)))
# XGBm_1=XGBClassifier(eval_metric='error',max_depth=11,alpha=0.6,subsample=0.65,colsample_bytree=0.7,colsample_bylevel = 0.8675692743597421,objective='binary:logistic',use_label_encoder=False,learning_rate=0.009,n_estimators=200,min_child_weight = 366,tree_method='hist',n_jobs=-1)
XGBm=XGBClassifier(eval_metric='error',max_depth=10,subsample=0.8,colsample_bytree=0.5,learning_rate=0.1,objective='binary:logistic',use_label_encoder=False,n_estimators=400,tree_method='hist')
XGBm_1=XGBClassifier(eval_metric='error',max_depth=12,subsample=1,colsample_bytree=0.8,learning_rate=0.25,objective='binary:logistic',use_label_encoder=False,n_estimators=200,tree_method='hist')
ADAm=AdaBoostClassifier(DecisionTreeClassifier(max_depth=9, min_samples_split=10, min_samples_leaf=5),
                         algorithm="SAMME",
                         n_estimators=600, learning_rate=1)
#ADAm_1=AdaBoostClassifier(DecisionTreeClassifier(max_depth=8, min_samples_split=10, min_samples_leaf=3),
#                          algorithm="SAMME",
#                          n_estimators=600, learning_rate=0.8)
GBm=GradientBoostingClassifier(max_depth=13, min_samples_split=10, min_samples_leaf=2,n_estimators=400,learning_rate=0.1,subsample=0.7)
ETm=ExtraTreesClassifier(n_estimators=400)
lr = LogisticRegression()
 
stackm = StackingClassifier(classifiers=[RFm,XGBm, XGBm_1,ADAm,GBm,ETm],use_probas=True,average_probas=False, meta_classifier=lr)
for clf, label in zip([RFm ,XGBm, XGBm_1, ADAm, GBm, ETm, stackm], 
 
                      ['RF', 

                       'XGB',
                       
                       'XGB',
                       
                       'ADA',
                       
                       'GB',
                       
                       'ET',
 
                       'Stack']):
 
 
 
    scores = model_selection.cross_val_score(clf, X1, y1,  cv=5, scoring='accuracy')
 
    print("Accuracy: %0.4f (+/- %0.4f) [%s]" % (scores.mean(), scores.std(), label))
    
    scores = model_selection.cross_val_score(clf, X1, y1,  cv=5, scoring='f1_macro')
 
    print("F1: %0.4f (+/- %0.4f) [%s]" % (scores.mean(), scores.std(), label))
    
'''
Accuracy: 0.9257 (+/- 0.0030) [RF]
F1: 0.8506 (+/- 0.0058) [RF]
Accuracy: 0.9302 (+/- 0.0021) [XGB]
F1: 0.8649 (+/- 0.0047) [XGB]
Accuracy: 0.9295 (+/- 0.0011) [XGB]
F1: 0.8645 (+/- 0.0024) [XGB]
Accuracy: 0.9293 (+/- 0.0023) [ADA]
F1: 0.8631 (+/- 0.0046) [ADA]
Accuracy: 0.9303 (+/- 0.0021) [GB]
F1: 0.8678 (+/- 0.0036) [GB]
Accuracy: 0.9297 (+/- 0.0024) [ET]
F1: 0.8582 (+/- 0.0071) [ET]
Accuracy: 0.9339 (+/- 0.0020) [Stack]
F1: 0.8709 (+/- 0.0036) [Stack]
'''


'''
Accuracy: 0.9240 (+/- 0.0027) [RF]
F1: 0.8482 (+/- 0.0064) [RF]
Accuracy: 0.9290 (+/- 0.0028) [XGB]
F1: 0.8628 (+/- 0.0060) [XGB]
Accuracy: 0.9296 (+/- 0.0023) [XGB]
F1: 0.8647 (+/- 0.0047) [XGB]
Accuracy: 0.9270 (+/- 0.0012) [ADA]
F1: 0.8590 (+/- 0.0042) [ADA]
Accuracy: 0.9302 (+/- 0.0017) [GB]
F1: 0.8654 (+/- 0.0041) [GB]
Accuracy: 0.9329 (+/- 0.0018) [ET]
F1: 0.8687 (+/- 0.0035) [ET]
Accuracy: 0.9321 (+/- 0.0022) [Stack]
F1: 0.8686 (+/- 0.0032) [Stack]
'''

In [31]:
df = pd.read_csv(r"C:\Users\admin\Desktop\kaggle\train.csv")
X1 = df.drop(['went_on_backorder','stop_auto_buy',"sku",'rev_stop','oe_constraint','potential_issue'],axis=1)
y1 = df['went_on_backorder']
# X1 = X.drop(['forecast_6_month'],axis=1)

In [32]:
X1

,national_inv,lead_time,in_transit_qty,forecast_3_month,forecast_6_month,forecast_9_month,sales_1_month,sales_3_month,sales_6_month,sales_9_month,min_bank,pieces_past_due,perf_6_month_avg,perf_12_month_avg,local_bo_qty,deck_risk,ppap_risk
0,4,2,0,0,0,0,0,0,0,0,0,0,0.18,0.31,0,1,0
1,2,16,0,0,0,0,0,0,0,0,1,0,0.31,0.40,0,0,0
2,4,4,0,0,0,0,0,0,0,0,0,0,0.73,0.78,0,0,0
3,0,9,0,1,1,1,0,0,0,0,0,0,0.86,0.69,0,0,0
4,300,8,0,0,0,0,0,40,100,122,7,0,0.00,0.00,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58914,11,8,0,0,0,0,2,6,17,26,6,0,0.54,0.64,0,1,0
58915,0,8,0,4,5,7,2,3,4,5,0,0,0.76,0.77,0,0,0
58916,0,2,0,1,1,1,0,0,0,0,0,0,0.09,0.06,0,0,0
58917,6,4,0,0,0,0,0,0,0,0,0,0,0.77,0.80,0,1,0


In [ ]:
RFm=RandomForestClassifier(criterion='gini',min_samples_leaf=1, max_depth=15,n_estimators=350)
# models.append(('GradientBoostingClassifier', GradientBoostingClassifier()))
# models.append(('AdaBoostClassifier', AdaBoostClassifier()))
# models.append(('MNBmodle', MultinomialNB()))
# models.append(('ExtraTreesClassifier', ExtraTreesClassifier()))
# mixmodels.append(('XGBClassifier', XGBClassifier(learning_rate=0.15, max_depth=14, n_estimators =200)))
# XGBm_1=XGBClassifier(eval_metric='error',max_depth=11,alpha=0.6,subsample=0.65,colsample_bytree=0.7,colsample_bylevel = 0.8675692743597421,objective='binary:logistic',use_label_encoder=False,learning_rate=0.009,n_estimators=200,min_child_weight = 366,tree_method='hist',n_jobs=-1)
XGBm=XGBClassifier(eval_metric='error',max_depth=10,subsample=0.8,colsample_bytree=0.5,learning_rate=0.1,objective='binary:logistic',use_label_encoder=False,n_estimators=400,tree_method='hist')
XGBm_1=XGBClassifier(eval_metric='error',max_depth=12,subsample=1,colsample_bytree=0.8,learning_rate=0.25,objective='binary:logistic',use_label_encoder=False,n_estimators=200,tree_method='hist')
ADAm=AdaBoostClassifier(DecisionTreeClassifier(max_depth=9, min_samples_split=10, min_samples_leaf=5),
                         algorithm="SAMME",
                         n_estimators=600, learning_rate=1)
#ADAm_1=AdaBoostClassifier(DecisionTreeClassifier(max_depth=8, min_samples_split=10, min_samples_leaf=3),
#                          algorithm="SAMME",
#                          n_estimators=600, learning_rate=0.8)
GBm=GradientBoostingClassifier(max_depth=13, min_samples_split=10, min_samples_leaf=2,n_estimators=400,learning_rate=0.1,subsample=0.7)
ETm=ExtraTreesClassifier(n_estimators=400)
lr = LogisticRegression()

stackm = StackingClassifier(classifiers=[RFm,XGBm, XGBm_1,ADAm,GBm,ETm],use_probas=True,average_probas=False, meta_classifier=lr)

In [17]:
from sklearn.pipeline import make_pipeline
from mlxtend.feature_selection import ColumnSelector

In [26]:
X1

,national_inv,lead_time,in_transit_qty,forecast_3_month,forecast_9_month,sales_1_month,sales_3_month,sales_6_month,sales_9_month,min_bank,pieces_past_due,perf_6_month_avg,perf_12_month_avg,local_bo_qty,deck_risk,ppap_risk
0,4,2,0,0,0,0,0,0,0,0,0,0.18,0.31,0,1,0
1,2,16,0,0,0,0,0,0,0,1,0,0.31,0.40,0,0,0
2,4,4,0,0,0,0,0,0,0,0,0,0.73,0.78,0,0,0
3,0,9,0,1,1,0,0,0,0,0,0,0.86,0.69,0,0,0
4,300,8,0,0,0,0,40,100,122,7,0,0.00,0.00,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58914,11,8,0,0,0,2,6,17,26,6,0,0.54,0.64,0,1,0
58915,0,8,0,4,7,2,3,4,5,0,0,0.76,0.77,0,0,0
58916,0,2,0,1,1,0,0,0,0,0,0,0.09,0.06,0,0,0
58917,6,4,0,0,0,0,0,0,0,0,0,0.77,0.80,0,1,0


In [ ]:
y1 = df_train['went_on_backorder']

In [ ]:
RFm=RandomForestClassifier(criterion='gini',min_samples_leaf=1, max_depth=17,n_estimators=400)
XGBm=XGBClassifier(eval_metric='error',max_depth=10,subsample=0.8,colsample_bytree=0.5,learning_rate=0.1,objective='binary:logistic',use_label_encoder=False,n_estimators=400,tree_method='hist')
XGBm_1=XGBClassifier(eval_metric='error',max_depth=12,subsample=1,colsample_bytree=0.8,learning_rate=0.25,objective='binary:logistic',use_label_encoder=False,n_estimators=200,tree_method='hist')
ADAm=AdaBoostClassifier(DecisionTreeClassifier(max_depth=9, min_samples_split=10, min_samples_leaf=5),
                         algorithm="SAMME",
                         n_estimators=600, learning_rate=1)
#ADAm_1=AdaBoostClassifier(DecisionTreeClassifier(max_depth=8, min_samples_split=10, min_samples_leaf=3),
#                          algorithm="SAMME",
#                          n_estimators=600, learning_rate=0.8)
GBm=GradientBoostingClassifier(max_depth=13, min_samples_split=10, min_samples_leaf=2,n_estimators=400,learning_rate=0.1,subsample=0.7)
ETm=ExtraTreesClassifier(n_estimators=400)
lr = LogisticRegression()

In [33]:
pipe1=[]
cols1=tuple([i for i in range(17) if i != 4])
pipe1.append(make_pipeline(ColumnSelector(cols=cols1),XGBm_1))
pipe1.append(make_pipeline(ColumnSelector(cols=cols1),GBm))
pipe1.append(make_pipeline(ColumnSelector(cols=cols1),ETm))

pipe2=[]
cols2=tuple([i for i in range(17)])
pipe2.append(make_pipeline(ColumnSelector(cols=cols2),XGBm))
pipe2.append(make_pipeline(ColumnSelector(cols=cols2),RFm))
pipe2.append(make_pipeline(ColumnSelector(cols=cols2),ADAm))


basem=pipe1+pipe2
basem

[Pipeline(steps=[('columnselector',
                  ColumnSelector(cols=(0, 1, 2, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13,
                                       14, 15, 16))),
                 ('xgbclassifier',
                  XGBClassifier(base_score=None, booster=None,
                                colsample_bylevel=None, colsample_bynode=None,
                                colsample_bytree=0.8, enable_categorical=False,
                                eval_metric='error', gamma=None, gpu_id=None,
                                importance_type=None,
                                interaction_constraints=None, learning_rate=0.2,
                                max_delta_step=None, max_depth=12,
                                min_child_weight=None, missing=nan,
                                monotone_constraints=None, n_estimators=600,
                                n_jobs=None, num_parallel_tree=None,
                                predictor=None, random_state=None,
          

In [ ]:
# XGB_2、GB、ET切去'forecast_6_month'

# 读入test
df = pd.read_csv('test.csv')
y = df['sku']

x = df.drop(["sku",'rev_stop','stop_auto_buy','oe_constraint','potential_issue'] , 1)
x1 = x.drop(['forecast_6_month'],axis=1)

sclf = StackingClassifier(classifiers=basem,use_probas=True,average_probas=False, meta_classifier=lr)
sclf.fit(X,y1)

y_pred = pd.DataFrame(sclf.predict(x))
# y_pred = pd.DataFrame(y_pred.drop([0], axis=1).values)
# y_pred = y_pred.rename(columns={ 0 : 'went_backorder_Prob'})
# y_pred_final = pd.concat([y,y_pred],axis=1)
y_pred_final

In [ ]:
y_pred = y_pred.rename(columns={ 0 : 'went_backorder_Prob'})
y_pred_final = pd.concat([y,y_pred],axis=1)
y_pred_final

In [ ]:
y_pred_final['predicted'] = y_pred_final.went_backorder_Prob.map( lambda x: 1 if x > 0.5 else 0)
# result=y_pred_final.drop(['went_on_backorder'], axis=1).values
result=y_pred_final.drop(['went_backorder_Prob'], axis=1).values
# x = df.drop(['went_on_backorder'], axis=1).values
from pandas import DataFrame
# print(str(len(y_pred_final)))
df = DataFrame(result,index=None,columns = ['sku','target'])
df.sort_values('sku', axis=0, ascending=True, inplace=True)
df.to_csv('sample.csv',header=0,index=0)

In [25]:
# lr = LogisticRegression(tol=0.0001, C=0.5, random_state=24, max_iter=10)
lr = LogisticRegression(C=0.5)
sclf = StackingClassifier(classifiers=basem,use_probas=True,average_probas=False, meta_classifier=lr)
for clf, label in zip([sclf], 
 
                      ['Stack']):
 
 
 
    scores = model_selection.cross_val_score(clf, X, y1,  cv=5, scoring='accuracy')
 
    print("Accuracy: %0.4f (+/- %0.4f) [%s]" % (scores.mean(), scores.std(), label))
    
#     scores = model_selection.cross_val_score(clf, X1, y1,  cv=5, scoring='f1_macro')
 
#     print("F1: %0.4f (+/- %0.4f) [%s]" % (scores.mean(), scores.std(), label))

Accuracy: 0.9332 (+/- 0.0021) [Stack]


In [24]:
lr = LogisticRegression(max_iter=200)
sclf = StackingClassifier(classifiers=basem,use_probas=True,average_probas=False, meta_classifier=lr)
for clf, label in zip([sclf], 
 
                      ['Stack']):
 
 
 
    scores = model_selection.cross_val_score(clf, X, y1,  cv=5, scoring='accuracy')
 
    print("Accuracy: %0.4f (+/- %0.4f) [%s]" % (scores.mean(), scores.std(), label))
    
    scores = model_selection.cross_val_score(clf, X, y1,  cv=5, scoring='f1_macro')
 
    print("F1: %0.4f (+/- %0.4f) [%s]" % (scores.mean(), scores.std(), label))

Accuracy: 0.9331 (+/- 0.0018) [Stack]
F1: 0.8721 (+/- 0.0052) [Stack]


In [34]:
lr = LogisticRegression(C=0.5)
sclf = StackingClassifier(classifiers=basem,use_probas=True,average_probas=False, meta_classifier=lr)
for clf, label in zip([sclf], 
 
                      ['Stack']):
 
 
 
    scores = model_selection.cross_val_score(clf, X, y1,  cv=5, scoring='accuracy')
 
    print("Accuracy: %0.4f (+/- %0.4f) [%s]" % (scores.mean(), scores.std(), label))
    
    scores = model_selection.cross_val_score(clf, X, y1,  cv=5, scoring='f1_macro')
 
    print("F1: %0.4f (+/- %0.4f) [%s]" % (scores.mean(), scores.std(), label))

Accuracy: 0.9333 (+/- 0.0024) [Stack]
F1: 0.8715 (+/- 0.0040) [Stack]


In [ ]:
lr = LogisticRegression(C=1.5,max_iter=400)
sclf = StackingClassifier(classifiers=basem,use_probas=True,average_probas=False, meta_classifier=lr)
for clf, label in zip([sclf], 
 
                      ['Stack']):
 
 
 
    scores = model_selection.cross_val_score(clf, X, y1,  cv=5, scoring='accuracy')
 
    print("Accuracy: %0.4f (+/- %0.4f) [%s]" % (scores.mean(), scores.std(), label))
    
    scores = model_selection.cross_val_score(clf, X, y1,  cv=5, scoring='f1_macro')
 
    print("F1: %0.4f (+/- %0.4f) [%s]" % (scores.mean(), scores.std(), label))

In [ ]:
lr = LogisticRegression(C=0.7)
sclf = StackingClassifier(classifiers=basem,use_probas=True,average_probas=False, meta_classifier=lr)
for clf, label in zip([sclf], 
 
                      ['Stack']):
 
 
 
    scores = model_selection.cross_val_score(clf, X1, y1,  cv=5, scoring='accuracy')
 
    print("Accuracy: %0.4f (+/- %0.4f) [%s]" % (scores.mean(), scores.std(), label))
    
#     scores = model_selection.cross_val_score(clf, X, y1,  cv=5, scoring='f1_macro')
 
#     print("F1: %0.4f (+/- %0.4f) [%s]" % (scores.mean(), scores.std(), label))

In [ ]:
lr = LogisticRegression(C=0.5,max_iter=200)
sclf = StackingClassifier(classifiers=basem,use_probas=True,average_probas=False, meta_classifier=lr)
for clf, label in zip([sclf], 
 
                      ['Stack']):
 
 
 
    scores = model_selection.cross_val_score(clf, X1, y1,  cv=5, scoring='accuracy')
 
    print("Accuracy: %0.4f (+/- %0.4f) [%s]" % (scores.mean(), scores.std(), label))
    
#     scores = model_selection.cross_val_score(clf, X, y1,  cv=5, scoring='f1_macro')
 
#     print("F1: %0.4f (+/- %0.4f) [%s]" % (scores.mean(), scores.std(), label))

In [ ]:
lr = LogisticRegression(C=0.3,max_iter=200)
sclf = StackingClassifier(classifiers=basem,use_probas=True,average_probas=False, meta_classifier=lr)
for clf, label in zip([sclf], 
 
                      ['Stack']):
 
 
 
    scores = model_selection.cross_val_score(clf, X, y1,  cv=5, scoring='accuracy')
 
    print("Accuracy: %0.4f (+/- %0.4f) [%s]" % (scores.mean(), scores.std(), label))
    
    scores = model_selection.cross_val_score(clf, X, y1,  cv=5, scoring='f1_macro')
 
    print("F1: %0.4f (+/- %0.4f) [%s]" % (scores.mean(), scores.std(), label))

In [ ]:
lr = LogisticRegression(C=0.7,max_iter=200)
sclf = StackingClassifier(classifiers=basem,use_probas=True,average_probas=False, meta_classifier=lr)
for clf, label in zip([sclf], 
 
                      ['Stack']):
 
 
 
    scores = model_selection.cross_val_score(clf, X1, y1,  cv=5, scoring='accuracy')
 
    print("Accuracy: %0.4f (+/- %0.4f) [%s]" % (scores.mean(), scores.std(), label))
    
#     scores = model_selection.cross_val_score(clf, X, y1,  cv=5, scoring='f1_macro')
 
#     print("F1: %0.4f (+/- %0.4f) [%s]" % (scores.mean(), scores.std(), label))

In [ ]:
lr = LogisticRegression(C=0.3,max_iter=400)
sclf = StackingClassifier(classifiers=basem,use_probas=True,average_probas=False, meta_classifier=lr)
for clf, label in zip([sclf], 
 
                      ['Stack']):
 
 
 
    scores = model_selection.cross_val_score(clf, X, y1,  cv=5, scoring='accuracy')
 
    print("Accuracy: %0.4f (+/- %0.4f) [%s]" % (scores.mean(), scores.std(), label))
    
    scores = model_selection.cross_val_score(clf, X, y1,  cv=5, scoring='f1_macro')
 
    print("F1: %0.4f (+/- %0.4f) [%s]" % (scores.mean(), scores.std(), label))

In [ ]:
lr = LogisticRegression()
sclf = StackingClassifier(classifiers=basem,use_probas=True,average_probas=False, meta_classifier=lr)
for clf, label in zip([sclf], 
 
                      ['Stack']):
 
 
 
    scores = model_selection.cross_val_score(clf, X1, y1,  cv=5, scoring='accuracy')
 
    print("Accuracy: %0.4f (+/- %0.4f) [%s]" % (scores.mean(), scores.std(), label))
    
    scores = model_selection.cross_val_score(clf, X1, y1,  cv=5, scoring='f1_macro')
 
    print("F1: %0.4f (+/- %0.4f) [%s]" % (scores.mean(), scores.std(), label))

In [ ]:
lr = LogisticRegression(C=0.5,max_iter=100)
sclf = StackingClassifier(classifiers=basem,use_probas=True,average_probas=False, meta_classifier=lr)
sclf.fit(X1,y1)

y_pred = pd.DataFrame(sclf.predict(x))
# y_pred = pd.DataFrame(y_pred.drop([0], axis=1).values)
# y_pred = y_pred.rename(columns={ 0 : 'went_backorder_Prob'})
# y_pred_final = pd.concat([y,y_pred],axis=1)
y_pred = y_pred.rename(columns={ 0 : 'went_backorder_Prob'})
y_pred_final = pd.concat([y,y_pred],axis=1)
y_pred_final['predicted'] = y_pred_final.went_backorder_Prob.map( lambda x: 1 if x > 0.5 else 0)
# result=y_pred_final.drop(['went_on_backorder'], axis=1).values
result=y_pred_final.drop(['went_backorder_Prob'], axis=1).values
# x = df.drop(['went_on_backorder'], axis=1).values
from pandas import DataFrame
# print(str(len(y_pred_final)))
df = DataFrame(result,index=None,columns = ['sku','target'])
df.sort_values('sku', axis=0, ascending=True, inplace=True)
df.to_csv('sample.csv',header=0,index=0)

In [ ]:
lr = LogisticRegression(C=0.7,max_iter=200)
sclf = StackingClassifier(classifiers=basem,use_probas=True,average_probas=False, meta_classifier=lr)
sclf.fit(X1,y1)

y_pred = pd.DataFrame(sclf.predict(x))
# y_pred = pd.DataFrame(y_pred.drop([0], axis=1).values)
# y_pred = y_pred.rename(columns={ 0 : 'went_backorder_Prob'})
# y_pred_final = pd.concat([y,y_pred],axis=1)
y_pred = y_pred.rename(columns={ 0 : 'went_backorder_Prob'})
y_pred_final = pd.concat([y,y_pred],axis=1)
y_pred_final['predicted'] = y_pred_final.went_backorder_Prob.map( lambda x: 1 if x > 0.5 else 0)
# result=y_pred_final.drop(['went_on_backorder'], axis=1).values
result=y_pred_final.drop(['went_backorder_Prob'], axis=1).values
# x = df.drop(['went_on_backorder'], axis=1).values
from pandas import DataFrame
# print(str(len(y_pred_final)))
df = DataFrame(result,index=None,columns = ['sku','target'])
df.sort_values('sku', axis=0, ascending=True, inplace=True)
df.to_csv('sample.csv',header=0,index=0)